In [0]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
wine_data = pd.read_csv('/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset/wine.csv')

In [3]:
wine_data.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
wine_data.shape

(178, 14)

In [5]:
wine_data.sample(8)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
83,1,13.05,3.86,2.32,22.5,85,1.65,1.59,0.61,1.62,4.80,0.84,2.01,515
106,1,12.25,1.73,2.12,19.0,80,1.65,2.03,0.37,1.63,3.40,1.00,3.17,510
141,2,13.36,2.56,2.35,20.0,89,1.40,0.50,0.37,0.64,5.60,0.70,2.47,780
27,0,13.30,1.72,2.14,17.0,94,2.40,2.19,0.27,1.35,3.95,1.02,2.77,1285
69,1,12.21,1.19,1.75,16.8,151,1.85,1.28,0.14,2.50,2.85,1.28,3.07,718
26,0,13.39,1.77,2.62,16.1,93,2.85,2.94,0.34,1.45,4.80,0.92,3.22,1195
143,2,13.62,4.95,2.35,20.0,92,2.00,0.80,0.47,1.02,4.40,0.91,2.05,550
82,1,12.08,1.13,2.51,24.0,78,2.00,1.58,0.40,1.40,2.20,1.31,2.72,630


In [6]:
wine_features = wine_data.drop('Class', axis=1)
wine_features.sample(8)

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
167,12.82,3.37,2.30,19.5,88,1.48,0.66,0.40,0.97,10.26,0.72,1.75,685
96,11.81,2.12,2.74,21.5,134,1.60,0.99,0.14,1.56,2.50,0.95,2.26,625
28,13.87,1.90,2.80,19.4,107,2.95,2.97,0.37,1.76,4.50,1.25,3.40,915
142,13.52,3.17,2.72,23.5,97,1.55,0.52,0.50,0.55,4.35,0.89,2.06,520
64,12.17,1.45,2.53,19.0,104,1.89,1.75,0.45,1.03,2.95,1.45,2.23,355
58,13.72,1.43,2.50,16.7,108,3.40,3.67,0.19,2.04,6.80,0.89,2.87,1285
33,13.76,1.53,2.70,19.5,132,2.95,2.74,0.50,1.35,5.40,1.25,3.00,1235
47,13.90,1.68,2.12,16.0,101,3.10,3.39,0.21,2.14,6.10,0.91,3.33,985


In [7]:
wine_target = wine_data[['Class']]
wine_target.sample(5)

,Class
100,1
110,1
99,1
142,2
24,0


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(wine_features, wine_target, test_size=0.4, random_state=0)

In [9]:
X_train.shape, Y_train.shape

((106, 13), (106, 1))

In [0]:
X_train = torch.from_numpy(X_train.values).float()
X_test = torch.from_numpy(X_test.values).float()

In [11]:
X_train.shape, X_test.shape

(torch.Size([106, 13]), torch.Size([72, 13]))

In [0]:
Y_train = torch.from_numpy(Y_train.values).view(1,-1)[0]
Y_test = torch.from_numpy(Y_test.values).view(1,-1)[0]

In [13]:
Y_train.shape, Y_test.shape

(torch.Size([106]), torch.Size([72]))

In [0]:
input_size = 13
output_size = 3
hidden_size = 100

In [0]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, output_size)

  def forward(self, X):
    X = torch.sigmoid(self.fc1(X))
    X = torch.sigmoid(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [0]:
model = Net()

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 0.01)

loss_fn = nn.NLLLoss()

In [19]:
cd '/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset'

/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset


In [22]:
!ls models

wine_checkpoint.pth  wine_classifier.pt  wine_classifier_state_dict


In [0]:
checkpoint = torch.load('models/wine_checkpoint.pth')

In [25]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [0]:
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [28]:
start_epoch = checkpoint['epoch']
start_epoch

99

In [29]:
loss = checkpoint['loss']
loss

tensor(0.5158, requires_grad=True)

In [30]:
model.train()

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

In [0]:
epochs = 500

In [32]:
for epoch in range(epochs):

  optimizer.zero_grad()
  Y_pred = model(X_train)

  loss = loss_fn(Y_pred, Y_train)
  loss.backward()

  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch : ', epoch, ' Loss : ', loss.item())

Epoch :  0  Loss :  0.5151585340499878
Epoch :  100  Loss :  0.07127352803945541
Epoch :  200  Loss :  0.05059628188610077
Epoch :  300  Loss :  0.046697117388248444
Epoch :  400  Loss :  0.044845640659332275


In [0]:
predict_out = model(X_test)
_, predict_y = torch.max(predict_out, 1)

In [0]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [35]:
print("Accuracy score : ", accuracy_score(Y_test.data, predict_y.data))
print("Precision score : ", precision_score(Y_test.data, predict_y.data, average='micro'))
print("Recall score : ", recall_score(Y_test.data, predict_y.data, average='micro'))

Accuracy score :  0.9444444444444444
Precision score :  0.9444444444444444
Recall score :  0.9444444444444444
